In [6]:
import pandas as pd
from database_clients import SQLiteClient
import os

In [2]:
sqlite = SQLiteClient("datawarehouse.sqlite")
sqlite_engine = sqlite.connect()

In [37]:
# sqlite.execute("UPDATE datacatalog SET data_inserted = False WHERE file_name = 'clickstream-dewiki-2017-11.tsv.gz'")
# sqlite.execute("UPDATE datacatalog SET size_in_bytes = 0 WHERE file_name = 'clickstream-dewiki-2017-11.tsv.gz'")
# sqlite.execute("UPDATE datacatalog SET number_of_rows = 0 WHERE file_name = 'clickstream-dewiki-2017-11.tsv.gz'")

In [3]:
datacatalog = pd.read_sql("SELECT * FROM datacatalog", sqlite_engine)
# clickstream = pd.read_sql("SELECT * FROM clickstream", sqlite_engine)

In [4]:
datacatalog.head()

,folder_name,file_name,resource_url,size_in_bytes,data_inserted,number_of_rows
0,2017-11,clickstream-dewiki-2017-11.tsv.gz,https://dumps.wikimedia.org/other/clickstream/...,55043649,1,4888887
1,2017-11,clickstream-enwiki-2017-11.tsv.gz,https://dumps.wikimedia.org/other/clickstream/...,311753414,1,25982518
2,2017-11,clickstream-eswiki-2017-11.tsv.gz,https://dumps.wikimedia.org/other/clickstream/...,35215170,1,3127869
3,2017-11,clickstream-jawiki-2017-11.tsv.gz,https://dumps.wikimedia.org/other/clickstream/...,21743549,1,2066173
4,2017-11,clickstream-ruwiki-2017-11.tsv.gz,https://dumps.wikimedia.org/other/clickstream/...,32058087,1,2507977


In [5]:
datacatalog.loc[datacatalog['folder_name'] == '2021-01', 'file_name']

404    clickstream-dewiki-2021-01.tsv.gz
405    clickstream-enwiki-2021-01.tsv.gz
406    clickstream-eswiki-2021-01.tsv.gz
407    clickstream-fawiki-2021-01.tsv.gz
408    clickstream-frwiki-2021-01.tsv.gz
409    clickstream-itwiki-2021-01.tsv.gz
410    clickstream-jawiki-2021-01.tsv.gz
411    clickstream-plwiki-2021-01.tsv.gz
412    clickstream-ptwiki-2021-01.tsv.gz
413    clickstream-ruwiki-2021-01.tsv.gz
414    clickstream-zhwiki-2021-01.tsv.gz
Name: file_name, dtype: object

In [40]:
file_path = os.path.join("data", "2021-01", "clickstream-dewiki-2021-01.tsv.gz")
columns = ["referrer", "resource", "type", "number_of_occurrences"]
# df = pd.read_csv(file_path, names=columns, sep='\t', compression='gzip', low_memory=False)
try:
    # df = pd.read_table(file_path, compression='gzip',  names=columns, low_memory=False)
    df = pd.read_table(file_path, compression='gzip', usecols= range(0,4), header=None, low_memory=False)
    df.columns = columns
except pd.errors.ParserError as err:
    print(err)
    print(f"File {file_path} is corrupt")
    find = 'saw '
    position = int(str(err).find(find)) + len(find)
    col_n = str(err)[position:]
    col_range = range(int(col_n))
    df = pd.read_table(file_path, compression='gzip',  names=col_range, low_memory=False)
    cols = range(len(columns))
    df = df.loc[:,cols]
    df.columns = columns

df.head()

,referrer,resource,type,number_of_occurrences
0,other-search,Ludwig_Maurer_(Unternehmer),external,14
1,Nile_(Band),At_the_Gate_of_Sethu,link,24
2,Schramberg,Schweizer_Electronic,link,21
3,other-empty,Schweizer_Electronic,external,29
4,Liste_der_börsennotierten_deutschen_Unternehmen,Schweizer_Electronic,link,12


In [36]:
# del df["XXX"]
# df.XXX.notna().sum()
df.shape

(5838404, 4)

In [30]:
datacatalog.shape

(503, 6)

In [31]:
datacatalog.dtypes

folder_name       object
file_name         object
resource_url      object
size_in_bytes      int64
data_inserted      int64
number_of_rows     int64
dtype: object

In [32]:
datacatalog.data_inserted.value_counts()

0    503
Name: data_inserted, dtype: int64

In [33]:
datacatalog[datacatalog["size_in_bytes"]> 0].shape[0]

109

In [34]:
sqlite.disconnect()